# STEP to X3D exporter
A jupyter notebook to export any STEP file (AP203, AP214, AP242) to X3D. The exporter is based upon pythonocc for topology and geometry processing and the x3d.py package. It is currently under development, authors Thomas Paviot (tpaviot@gmail.com) and Andreas Plesch (gh @andreasplesch). The code is distributed under the terms of the LGPL V3 public license.

Step 1. Import all necessary packages and modules

In [ ]:
# standard imports
import os
import tempfile

# pythonocc imports
from OCC.Extend.DataExchange.XDE import SceneGraphFromDoc, DocFromSTEP
from OCC.Extend.DataExchange.X3D import X3DShapeExporter, X3DFromSceneGraph

# ipython and ipywidgets imports
from ipywidgets import FileUpload
from IPython.display import HTML, display, FileLink

Step 2. Pick a STEP file from your computer. Allowed extensions are .stp, .step, .STP, .STEP. Max size is restricted to 10Mb. The "Pick a STEP file" button will turn into light green when upload is completed. Untill it remains yellow, the upload is not achieved or it failed.

In [ ]:
tmp_dir = tempfile.mkdtemp()  # files are stored to a temporary directory
base_filename = None
full_filename = None

def _handle_upload(change):
    global base_filename, full_filename
    w = change['owner']
    base_filename = next(iter(file_uploader.value))
    full_filename = os.path.join(tmp_dir, base_filename)
    content = file_uploader.value[base_filename]['content']
    with open(full_filename, 'wb') as f:
        f.write(content)
    print('Uploaded `{}` ({:.2f} kB)'.format(full_filename, len(content) / 2**10))
    file_uploader.style.button_color = "lightgreen"
    
file_uploader = FileUpload(description="Pick a STEP file on your machine",
                accept='.stp, .step, .STEP, .STP', multiple=False)
file_uploader.style.button_color = "yellow"
file_uploader.observe(_handle_upload, names='data')
file_uploader

Step 3. Process the STEP file, generates a scenegraph and an x3dom HTML export

In [ ]:
document = DocFromSTEP(full_filename).get_doc()
scenegraph = SceneGraphFromDoc(document)
x3dXML = X3DFromSceneGraph(scenegraph.get_scene(), scenegraph.get_internalFaceEntries())
x3dom_html = x3dXML.to_x3dom_html()

Render the x3dom hml in the jupyter notebook

In [ ]:
HTML(x3dom_html)

Enable downloading the X3D output as a .x3d file

In [ ]:
# write the X3D file to the disk
pre, ext = os.path.splitext(base_filename)
x3dfilename = pre + ".x3d"

with open(x3dfilename, 'w') as f:
    f.write(x3dXML.to_xml())

local_file = FileLink(x3dfilename, result_html_prefix="Click here to download: ")
display(local_file)